Import Data

In [ ]:
import pandas as pd

# Load CSV files
transactions = pd.read_csv('Data/transactions.csv')
account_holders = pd.read_excel('Data/account_holders.xlsx')

# Load data from APIs
import requests

def fetch_exchange_rates(api_key):
    url = f'https://openexchangerates.org/api/latest.json?app_id={api_key}'
    response = requests.get(url)
    return response.json()

def fetch_market_data(api_key, symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    return response.json()

exchange_rates = fetch_exchange_rates('your_exchange_rates_api_key')
market_data = fetch_market_data('your_alpha_vantage_api_key', 'AAPL')


Data Cleaning and Merging

In [ ]:
# Handling missing values
transactions.dropna(inplace=True)
account_holders.dropna(inplace=True)

# Correcting data types
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
account_holders['account_creation_date'] = pd.to_datetime(account_holders['account_creation_date'])

# Merging datasets
merged_data = pd.merge(transactions, account_holders, on='account_id', how='inner')



Calculating New Values

In [ ]:
# Calculate average transaction amount and transaction frequency
merged_data['avg_transaction_amount'] = merged_data.groupby('account_id')['transaction_amount'].transform('mean')
merged_data['transaction_frequency'] = merged_data.groupby('account_id')['transaction_id'].transform('count')


Store Clean Data in SQLite Database

In [ ]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
db_path = 'Data/fraud_detection.db'
conn = sqlite3.connect(db_path)

# Save the cleaned DataFrame to a SQLite table
merged_data.to_sql('transactions', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print(f"Database created and stored at {db_path}")

Querying Data from SQLite for Visualization

In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect(db_path)

# Define SQL queries
query_trend = """
SELECT transaction_date, SUM(transaction_amount) as total_amount
FROM transactions
GROUP BY transaction_date
ORDER BY transaction_date;
"""

query_account_comparison = """
SELECT account_id, SUM(transaction_amount) as total_amount
FROM transactions
GROUP BY account_id
ORDER BY total_amount DESC;
"""

query_correlation = """
SELECT transaction_frequency, avg_transaction_amount
FROM transactions;
"""

# Execute queries and store results in DataFrames
trend_data = pd.read_sql_query(query_trend, conn)
account_comparison_data = pd.read_sql_query(query_account_comparison, conn)
correlation_data = pd.read_sql_query(query_correlation, conn)

# Close the connection
conn.close()


Visualiziing Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Line plot for transaction trends over time
plt.figure(figsize=(10, 6))
sns.lineplot(data=trend_data, x='transaction_date', y='total_amount')
plt.title('Transaction Trends Over Time')
plt.show()

# Bar chart comparing transaction amounts by account
plt.figure(figsize=(10, 6))
sns.barplot(data=account_comparison_data, x='account_id', y='total_amount')
plt.title('Transaction Amounts by Account')
plt.xticks(rotation=90)
plt.show()

# Scatter plot showing the correlation between transaction frequency and amount
plt.figure(figsize=(10, 6))
sns.scatterplot(data=correlation_data, x='transaction_frequency', y='avg_transaction_amount')
plt.title('Correlation between Transaction Frequency and Amount')
plt.show()


Output To CSV for Visualizations

In [ ]:
trend_data.to_csv('Data/Out/trend_data.csv', index=False)
account_comparison_data.to_csv('Data/Out/account_comparison_data.csv', index=False)
correlation_data.to_csv('Data/Out/correlation_data.csv', index=False)



Unit Test

In [ ]:
import unittest

class TestDataCleaning(unittest.TestCase):
    def test_dropna(self):
        data = pd.DataFrame({'A': [1, 2, None], 'B': [4, None, 6]})
        cleaned_data = data.dropna()
        self.assertEqual(cleaned_data.shape[0], 1)
    
    def test_correct_data_type(self):
        data = pd.DataFrame({'date': ['2021-01-01', '2021-01-02']})
        data['date'] = pd.to_datetime(data['date'])
        self.assertTrue(pd.api.types.is_datetime64_any_dtype(data['date']))

if __name__ == '__main__':
    unittest.main()
